In [1]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import seaborn as sns # SEABORN
import scipy
import sklearn
import random as rd
from sklearn.decomposition import PCA
from IPython.display import display
import os 
from pathlib import Path
import shutil 


In [ ]:
Assumptions :
Each sequence displays one of the six basic emotional ex-
pressions.

The first frame of each sequence shows an expression-neutral face.

The intensity of the expression increases over the
frames.(others dont say the final is 1 .We do)

In [ ]:
we need to understansd the landmarks (annotaste) and the aam for the test image (input images)

In [ ]:
we have todo procustres and we need noormalization of the landmark differences,